# Generating Counterfactuals

In this notebook, we will focus on generating counterfactuals from individual
datapoints. This will be implemented for the following models:

- Naive Bayes
- Fair Bayesian Network
- Fair Random Forest Classifier

By generating counterfactuals, we hope to gain insight into how the model uses
the different attributes in it's decisions.

In [1]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

from forseti.bayesnet import latentLabelClassifier, interpretableNaiveBayes
import pandas as pd

df = pd.read_csv('data/adult.csv')
    
clf = interpretableNaiveBayes()

tmp = df
label = 'income'

clf.train(
    label,
    tmp,
    'NB'
)

In [2]:
X = clf.generateCounterfactuals()
X

/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:537: UserWarning: Found unknown state name. Trying to switch to using all state names as state numbers
  UserWarning,
/home/bcwein/anaconda3/envs/forseti/lib/python3.7/site-packages/pgmpy/factors/discrete/DiscreteFactor.py:472: RuntimeWarning: invalid value encountered in true_divide
  phi.values = phi.values / phi.values.sum()


,age,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,hours-per-week,O1,O2,O3,O4
5031,0,3,11,0,2,1,4,1,0,2,1.000000,0.9,1,0.0
5031,1,3,11,0,2,1,4,0,0,2,1.000000,0.9,1,0.0
5031,1,3,11,2,2,1,4,1,0,2,1.000000,0.9,1,0.0
5031,1,3,11,0,2,1,4,1,0,4,1.000000,0.9,1,0.0
5031,1,3,11,0,2,1,4,1,0,2,1.000000,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5031,1,4,11,0,2,1,4,1,0,2,0.864943,0.9,1,0.2
5031,1,3,11,0,2,1,4,1,0,0,1.000000,0.9,1,0.2
5031,1,3,11,0,2,1,4,1,2,2,1.000000,0.9,1,0.2
5031,1,3,11,0,1,1,4,1,0,2,1.000000,0.9,1,0.2
